In [5]:
'''COPY PASTED CORE ON 05/02/2025 FROM ROUND-BENEDETTO'''
'''UPDATE AS NEEDED'''
import numpy as np
from numpy import random as npr
from scipy.special import comb
import itertools
cooperate = np.array( [ 1, 0 ] )
defect = np.array( [ 0, 1 ] )
# defining all strategies
# # template function
# parameter_list = [ k, ]
# def strategy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) : 
#     # move either cooperate or defect
#     move
#     return move 
# dictionary of all strategies

# default Payoff matrix
default_R = 2
default_S = 0
default_T = 3
default_P = 1
Payoff = np.array( [ [ default_R , default_S ], [ default_T, default_P ] ] )

# STRATEGIES:________________________________________________
    # template function:
    # def strategy( round_number = 0, match_history = [[]], player_index = 0 ) : 
    #     # move either cooperate or defect
    #     move = cooperate/defect
    #     return move 

# nice guy function: always cooperate
def nice_guy( round_number = 0, match_history = [[]], player_index = 0 ) :      # 'None' al posto di [[]] ?
    move = cooperate
    return move 

# bad guy function: always defect 
def bad_guy( round_number = 0, match_history = [[]], player_index = 0 ) :        # 'None' al posto di [[]] ?
    move = defect
    return move 

# mainly nice guy: randomly defect k% of the times and cooperate 100-k %, k<50
def mainly_nice( round_number = 0, match_history = [[]], player_index = 0 ) :                   # eccezione da gestire meglio?
    k = 0.4    # avoiding global variable dependace
    u = npr.random()                                                                           # corretto in random() invece di random(1)
    if ( u < k ) : move = defect
    else : move = cooperate
    return move

# mainly bad guy: randomly defect k% of the times and cooperate 100-k %, k>50
def mainly_bad( round_number = 0, match_history = [[]], player_index = 0 ) :                                                                           # eccezione da gestire meglio?
    k = 0.6
    u = npr.random()                                                                            # corretto in random() invece di random(1)
    if ( u < 1-k ) : move = defect
    else : move = cooperate
    return move

# random guy: randomly defect or cooperate
def random_guy( round_number = 0, match_history = [[]], player_index = 0 ) :                                                                            # eccezione da gestire meglio?
    u = npr.random()                                                                           # corretto in random() invece di random(1)
    if ( u < 0.5 ) : move = defect
    else : move = cooperate
    return move

# tit_for_tat function: start by cooperating, then repeat what the opponent has done in the previous move
def tit_for_tat( round_number = 0, match_history = [[]], player_index = 0 ) : 
    if ( round_number == 0 ) : move = cooperate
    else : move = match_history[ round_number - 1 ][ 1 - player_index ]
    return move

# resentful guy: cooperate until the opponent defects once, then always defect
def resentful_guy( round_number = 0, match_history = [[]], player_index = 0 ) : 
    move = cooperate
    if ( round_number > 0 ) :
        for res in match_history :
            if np.all( res[ 1 - player_index  ] == defect ) :
                move = defect
    return move

# trusting guy (increasing odds of cooperating according to match history):
# initially is like random guy (coop/defect at k=50%), then updates k
def trusting_guy( round_number = 0, match_history = [[]], player_index = 0 ) :
    k = 0.5     # initial trust
    # environmental trust
    if ( round_number > 0 ) :
        coop = 0.         # numero di cooperazioni avversarie finora
        # 'res[ 1 - player_index  ][ 0 ]' prende il primo elemento della mossa avversaria (1 se ha cooperato, 0 se ha defectato)
        for res in match_history : coop += res[ 1 - player_index  ][ 0 ]
        k = coop / len( match_history )           # updated k
    u = npr.random()                                                                          # corretto in random() invece di random(1)
    if ( u > k ) : move = defect
    else : move = cooperate
    return move

# balancing guy (cooperates only for an even cooperation history, otherwise defect)
def balancing_guy( round_number = 0, match_history = [[]], player_index = 0 ) :
    move = defect
    if ( round_number > 0 ) :
        # total cooperation seen
        coop = 0
        for res in match_history : coop += ( res[ 0 ][ 0 ] + res[ 1 ][ 0 ])
        if ( ( coop % 2 ) == 0 ) : move = cooperate
    return move

# Mid resentful: cooperates on the first move, and defects if the opponent has defected on any of the previous 3 moves, else cooperates
def mid_resentful( round_number = 0, match_history = [[]], player_index = 0):
    move = cooperate
    # checko gli ultimi 3 turni (o meno se sono all'inizio):
    start = max(0, round_number - 3)
    for round_i in range(start, round_number):
        # controllo se la mossa avversaria corrisponde ad un defect=[0,,1]
        if np.array_equal(match_history[round_i][1 - player_index], defect):   
            move = defect
            break  # non serve continuare il controllo, esco dal for
    return move

# reverse tit_for_tat function: start by defecting, then does the opposite of what the opponent has done in the previous move
def reverse_tit_for_tat( round_number = 0, match_history = [[]], player_index = 0 ) : 
    if ( round_number == 0 ) : move = defect
    else : move = [1,1] - match_history[ round_number - 1 ][ 1 - player_index ]  
            # in this way i obtain the opposite move: [1,1] - [1,0] = [0,1] and [1,1] - [0,1] = [1,0]
    return move

strategies = {
    'NiceGuy' : nice_guy,
    'BadGuy' : bad_guy,
    'MainlyNice' : mainly_nice,
    'MainlyBad' : mainly_bad,
    'RandomGuy': random_guy,
    'TitForTat' : tit_for_tat,
    'ResentfulGuy' : resentful_guy,
    'TrustingGuy' : trusting_guy,
    'Thanos' : balancing_guy,
    'MidResentful': mid_resentful,
    'ReverseTft': reverse_tit_for_tat,
}

# GAMES FUNCTIONS:

# function to play one round 1 vs 1 for two different strategies players
def round( key_1, key_2, round_number=0, match_history=[[]], M = Payoff ) :
    # getting strategies from dictionary
    S_1 = strategies[ key_1 ]
    S_2 = strategies[ key_2 ]
    # computing next move for each player
    u_1 = S_1( round_number, match_history, 0 )
    u_2 = S_2( round_number, match_history, 1 )
    # computing rewards for each player
    r_1 = np.dot( u_1, Payoff.dot(u_2))
    r_2 = np.dot( u_2, Payoff.dot(u_1))
    # updating match history
    match_history.append( [ u_1, u_2 ] )
    # returning results
    return r_1, r_2



# function to play a match of N rounds 1 vs 1 for two different strategies player
def match( key_1, key_2, N_rounds = 10, M = Payoff ) : 
    # match history strarting empty
    history = [ ]                                                                           
    # starting total rewards
    R_1 = 0
    R_2 = 0
    # list of partial sums
    Rewards = []
    for round_i in range( 0, N_rounds ) :
        r_1, r_2 = round( key_1, key_2, round_i, history, M )
        R_1 += r_1
        R_2 += r_2
        Rewards.append( [ R_1, R_2 ] )
    return Rewards              # it returns a list wiith the partial sums of the rewards

In [13]:
def tourney(participants={},N_gironi=1,N_rounds = 10, M = Payoff):
    # Retrieve only the names of the strategies repeated as many times as their frequency allows.
    player_list_from_dict = []
    for item in (list(participants.keys())):
        for i in range (participants[item]):
            player_list_from_dict.append(item)
    # Compute the combinations
    b = list(itertools.combinations(player_list_from_dict, 2))
    N_giornata = len(b)
    # List which will be used to have the scores of each match by the side of the two opponents
    list_with_scores = b
    # Loop over the combinations one by one
    for i in range (N_giornata):
        # Player1 and Player2 strategies
        player1 = b[i][0]
        player2 = b[i][1]
        # Initiliaze an empty list to store the results over the match
        AllRewards = []
        # Do the match between the two players and update the rewards; The ordered syntax for match is:
        # match( key_1, key_2, N_rounds = 10, M = Payoff ) : 
        AllRewards =  match(player1, player2, N_rounds, M)
        # Retrieve the final scores for Player1 and Player2 as the last element in the list
        R1, R2 = AllRewards[-1][0], AllRewards[-1][1]
        list_with_scores[i] += (R1,R2)
    # Un-comment the following print to debug
    #for item in list_with_scores: print(item)
    
    # Good, now each combination has the corresponding score: we now need to assign it to the respective players and calculate the final score
    strat = list(participants.keys())
    freq = [participants[key] for key in strat]
    named =[]
    # Add identification to the layer trough the use of a numerical value from 1 up to their frequency BUT PRESERVE THE ORDER
    for key,f in zip (strat,freq):
        for j in range(1,f+1):
            if (f!=0):
                named.append(key + str(j))
    # Compute the permutations preserving the order
    with_names = list(itertools.combinations(named,2))
    # Copy the scores for each match over to the ones with identification
    # In list_with_scores[I][J] we have: I = ordered number of the combination, and J:
    # J=0 is 1st strategy; J=1 is 2nd strategy; J=2 is Player1 score; J=3 is Player2 score
    for i in range (N_giornata): with_names[i] += (list_with_scores[i][2],list_with_scores[i][3] )
    # Un-comment the following print to debug
    #for item in with_names: print(item)

    # Compute the score for each named player and add it to the list
    FinalScores = []
    for player in named:
        SumOver1 = np.sum([ with_names[i][2] for i in range (N_giornata) if (with_names[i][0] == player) ])
        SumOver2 = np.sum([ with_names[i][3] for i in range (N_giornata) if (with_names[i][1] == player) ])
        FinalScores.append(SumOver1 + SumOver2)
    Results = named
    
    for i in range (len(Results)):
        Results[i] = (Results[i],FinalScores[i])

    return ( Results )    

In [14]:
# Tourney test
test_part ={ 
    'NiceGuy' : 3,
    'BadGuy' : 3,
    'MainlyNice' : 4,
    'MainlyBad' : 3,
    'RandomGuy' : 1,
    'TitForTat' : 1,
    'ResentfulGuy' : 2,
    'TrustingGuy' : 1,
    'Thanos' : 2,
    'MidResentful' : 1,
    'ReverseTft': 2,
}

result = tourney(test_part,N_gironi=1,N_rounds = 10, M = Payoff)
for item in result: print(item)
print("\nHere's the Ranking at the end of the tourney:")
# Sorted gives a copy, while using LIST.sort() does it in place.
Ranking = sorted(result,reverse=True, key=lambda x: x[1])
for item in Ranking: print(item)

('NiceGuy1', 276.0)
('NiceGuy2', 262)
('NiceGuy3', 256)
('BadGuy1', 430)
('BadGuy2', 416)
('BadGuy3', 430)
('MainlyNice1', 315)
('MainlyNice2', 335)
('MainlyNice3', 285)
('MainlyNice4', 320)
('MainlyBad1', 294)
('MainlyBad2', 301)
('MainlyBad3', 327)
('RandomGuy1', 328)
('TitForTat1', 327)
('ResentfulGuy1', 388)
('ResentfulGuy2', 377)
('TrustingGuy1', 344)
('Thanos1', 332)
('Thanos2', 336)
('MidResentful1', 363)
('ReverseTft1', 323)
('ReverseTft2', 323.0)

Here's the Ranking at the end of the tourney:
('BadGuy1', 430)
('BadGuy3', 430)
('BadGuy2', 416)
('ResentfulGuy1', 388)
('ResentfulGuy2', 377)
('MidResentful1', 363)
('TrustingGuy1', 344)
('Thanos2', 336)
('MainlyNice2', 335)
('Thanos1', 332)
('RandomGuy1', 328)
('MainlyBad3', 327)
('TitForTat1', 327)
('ReverseTft1', 323)
('ReverseTft2', 323.0)
('MainlyNice4', 320)
('MainlyNice1', 315)
('MainlyBad2', 301)
('MainlyBad1', 294)
('MainlyNice3', 285)
('NiceGuy1', 276.0)
('NiceGuy2', 262)
('NiceGuy3', 256)


In [15]:
print("Data format is as follows:")
print("First index is the default position in the list, Result[0]: ",result[0])
print("Second index is [0] for the strategy/player_name and [1] for its reward:")
print("Result[0][0]: ",result[0][0], "       Result[0][1]: " ,result[0][1])

Data format is as follows:
First index is the default position in the list, Result[0]:  ('NiceGuy1', 276.0)
Second index is [0] for the strategy/player_name and [1] for its reward:
Result[0][0]:  NiceGuy1        Result[0][1]:  276.0


In [16]:
labels = list(test_part.keys())
players = [ result[i][0] for i in range (len(result)) ]
rewards = [ result[i][1] for i in range (len(result)) ]
print(labels)
print(players)
mean_rewards= []
index = 0
for item in labels:
    # Make it a string to apply the count method: returns the number of times a substring is included in the main string
    f = str(players).count(item)
    # Compute the mean of their rewards
    mean = (np.sum(rewards[index:index+f]))/f
    mean_rewards.append(mean)
    index += f
for item in zip(labels,mean_rewards): print(item)

['NiceGuy', 'BadGuy', 'MainlyNice', 'MainlyBad', 'RandomGuy', 'TitForTat', 'ResentfulGuy', 'TrustingGuy', 'Thanos', 'MidResentful', 'ReverseTft']
['NiceGuy1', 'NiceGuy2', 'NiceGuy3', 'BadGuy1', 'BadGuy2', 'BadGuy3', 'MainlyNice1', 'MainlyNice2', 'MainlyNice3', 'MainlyNice4', 'MainlyBad1', 'MainlyBad2', 'MainlyBad3', 'RandomGuy1', 'TitForTat1', 'ResentfulGuy1', 'ResentfulGuy2', 'TrustingGuy1', 'Thanos1', 'Thanos2', 'MidResentful1', 'ReverseTft1', 'ReverseTft2']
('NiceGuy', 264.6666666666667)
('BadGuy', 425.3333333333333)
('MainlyNice', 313.75)
('MainlyBad', 307.3333333333333)
('RandomGuy', 328.0)
('TitForTat', 327.0)
('ResentfulGuy', 382.5)
('TrustingGuy', 344.0)
('Thanos', 334.0)
('MidResentful', 363.0)
('ReverseTft', 323.0)
